In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install flask line-bot-sdk flask-ngrok

     |████████████████████████████████| 71kB 3.2MB/s 


In [ ]:
'''
解壓縮模型
'''
from zipfile import ZipFile

with ZipFile('converted_savedmodel.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('converted_savedmodel')

In [ ]:
'''
載入類別列表
'''
class_dict = {}
with open('converted_savedmodel/labels.txt') as f:
    for line in f:
       (key, val) = line.split()
       class_dict[int(key)] = val

In [ ]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, FollowEvent,RichMenu,TemplateSendMessage,MessageTemplateAction
)

from flask_ngrok import run_with_ngrok

from linebot.models.template import ConfirmTemplate

import json


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 創建一個LineBotAPI
# LineBotApi(channel_access_token)
# 後續都會拿 line_bot_api 跟line溝通
line_bot_api = LineBotApi("R2SYuUgAqgEqK9dcsFk+tpkXVek/6Uyz1u7QLuVheAUZ0cnOkStae7Prx1QpCbZ0LzAodMPpBHDTRms1b7x0UhTXpuwJwyC9XDLYSEgcwLU6GHUYBJ660hDj5WlnPRrXbcwCWlWzgGrkxqrN34h0MQdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler('c9329d4e3aa6586f7d287261523314c0')

@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

In [ ]:
@handler.add(FollowEvent)
def process_follow_event(event):
  # 取出消息內User的資料
  user_profile=line_bot_api.get_profile(event.source.user_id)
  with open("/content/drive/MyDrive/users.txt", "a") as file:
    file.write(json.dumps(vars(user_profile),sort_keys=True))
    file.write('\n')
  # 回覆文字消息與圖片消息
  # event.reply_token=收到這條訊息要回應的對象,event.reply_token是一次性的
  line_bot_api.reply_message(
      event.reply_token,
      TextSendMessage("Hello,"+user_profile.display_name+"\n歡迎來到Lakers Zone")
  )


In [ ]:

#載入前面的圖文選單設定，
#由於已經有json檔案所以用的是load()
#並要求line_bot_api將圖文選單上傳至Line
#最後會得到richmenu的Id
with open('richmenu.json','r',encoding='utf8') as file:
  richmenu_config=json.load(file)
lineRichMenuId = line_bot_api.create_rich_menu(rich_menu=RichMenu.new_from_json_dict(richmenu_config))
print(lineRichMenuId)

richmenu-1c5df5b7f6a59d2bc81695d466580436


In [ ]:
#將先前準備的菜單照片，以Post消息寄發給Line

#    載入照片
#    要求line_bot_api，將圖片傳到先前的圖文選單id
uploadImage=open('/content/menupicture.png','rb')
setImageResponse = line_bot_api.set_rich_menu_image(lineRichMenuId,'image/png',uploadImage)
print(setImageResponse)

None


In [ ]:
with open("/content/drive/MyDrive/users.txt","r",encoding="utf8") as file:
  for x in file.readlines():
    user_info=json.loads(x)
userid=user_info["user_id"]
#  x=file.read()
#user_info=json.loads(x)
linkResult=line_bot_api.link_rich_menu_to_user( userid, lineRichMenuId)
print(linkResult)

None


In [ ]:
#用戶發出文字消息時， 按條件內容, 回傳文字消息
#當handler(總機)收到訊息事件(MessageEvent)且為TextMessage時就做下面的方法
#get()函数返回指定键的值
#find()如果找不到會回傳
@handler.add(MessageEvent,message=TextMessage)
def process_message_event(event):
  reply_arr=[]
  reply_arr.append(TextSendMessage("請不要罵髒話"))
  reply_arr.append(TextSendMessage("我把你的選單解除掉了\n請離開!~"))
  textmessage=event.message.text
  if( '幹' in textmessage!=False or 'Fuck' in textmessage!=False):
      line_bot_api.reply_message(
        event.reply_token,
        reply_arr
      )
      lineUnregisterUserMenuResponse=line_bot_api.unlink_rich_menu_from_user(userid)
  elif textmessage=="球員圖像辨識":
    Confirm_template = TemplateSendMessage(
        alt_text='目錄 template',
        template=ConfirmTemplate(
            title='這是ConfirmTemplate',
            text='您確定要執行球員影像辨識嗎？',
            actions=[                              
                MessageTemplateAction(
                    label='是',
                    text='是我要執行'
                ),
                MessageTemplateAction(
                    label='否',
                    text='否我不要執行'
                )
            ]
        )
    )
    line_bot_api.reply_message(event.reply_token,Confirm_template)
  elif '對不起' in textmessage!=False:
     line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage("好吧!我考慮看看")
      )



In [ ]:
'''

圖片消息，解析圖片

'''

import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np

# 引用套件
from linebot.models import (
    MessageEvent,ImageMessage,TextSendMessage
)

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = tensorflow.keras.models.load_model('converted_savedmodel/model.savedmodel',compile=False)

import time

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):

    print(time.asctime( time.localtime(time.time()) ))

    message_content = line_bot_api.get_message_content(event.message.id)
    file_name = event.message.id+'.jpg'
    with open(file_name, 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

    print(time.asctime( time.localtime(time.time()) ))

    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Replace this with the path to your image
    image = Image.open(file_name)

    #resize the image to a 224x224 with the same strategy as in TM2:
    #resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    print(time.asctime( time.localtime(time.time()) ))
    
    #turn the image into a numpy array
    image_array = np.asarray(image)

    # display the resized image
    image.show()

    
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0 - 1 )

    # Load the image into the array
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0]= normalized_image_array[0:224,0:224,0:3]

    # run the inference
    prediction = model.predict(data)

    print(time.asctime( time.localtime(time.time()) ))

    max_probability_item_index = np.argmax(prediction[0])

    if prediction.max() > 0.9997:
        line_bot_api.reply_message(
          event.reply_token,
          TextSendMessage(
          """這個照片應該是 %s ，其相似機率為 %s """ %(class_dict.get(max_probability_item_index), prediction[0][max_probability_item_index])
          )
        )
    else :
      line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(
        """這張照片既不是Lebron也不是Davis!"""
        )
      )

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d42d9c18ab24.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
